In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import time
from datetime import datetime

In [3]:
#Extract data from CSV
df1=pd.read_csv("C:/Users/teddy/Desktop/database.csv")

In [4]:
epoch = datetime(1970, 1, 1)

def mapdateTotime(x):
    try:
        dt = datetime.strptime(x, "%m/%d/%Y")
    except ValueError:
        dt = datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
    diff = dt - epoch
    return diff.total_seconds()

df1.Date = df1.Date.apply(mapdateTotime)

In [5]:
col1 = df1[['Date','Latitude','Longitude','Depth']]
col2 = df1['Magnitude']
#Convert to Numpy array
InputX1 = col1.as_matrix()
InputY1 = col2.as_matrix()
print(InputX1)

[[ -1.57680000e+08   1.92460000e+01   1.45616000e+02   1.31600000e+02]
 [ -1.57507200e+08   1.86300000e+00   1.27352000e+02   8.00000000e+01]
 [ -1.57420800e+08  -2.05790000e+01  -1.73972000e+02   2.00000000e+01]
 ..., 
 [  1.48288320e+09   3.69179000e+01   1.40426200e+02   1.00000000e+01]
 [  1.48296960e+09  -9.02830000e+00   1.18663900e+02   7.90000000e+01]
 [  1.48305600e+09   3.73973000e+01   1.41410300e+02   1.19400000e+01]]


In [6]:
#Min-max Normalization
X1_min = np.amin(InputX1,0)     
X1_max = np.amax(InputX1,0)   
print("Mininum values:",X1_min)
print("Maximum values:",X1_max)
Y1_min = np.amin(InputY1)     
Y1_max = np.amax(InputY1) 
InputX1_norm = (InputX1-X1_min)/(X1_max-X1_min)
InputY1_norm = InputY1  #No normalization in output

#Reshape
Xfeatures = 3 #Number of input features
Yfeatures = 1 #Number of input features
samples = 23000 # Number of samples

InputX1_reshape = np.resize(InputX1_norm,(samples,Xfeatures))
InputY1_reshape = np.resize(InputY1_norm,(samples,Yfeatures))

Mininum values: [ -1.57680000e+08  -7.70800000e+01  -1.79997000e+02  -1.10000000e+00]
Maximum values: [  1.48305600e+09   8.60050000e+01   1.79998000e+02   7.00000000e+02]


In [7]:
#Training data
batch_size = 2000
InputX1train = InputX1_reshape[0:batch_size,:]
InputY1train = InputY1_reshape[0:batch_size,:]
#Validation data
v_size = 2500
InputX1v = InputX1_reshape[batch_size:batch_size+v_size,:]
InputY1v = InputY1_reshape[batch_size:batch_size+v_size,:]

In [8]:
learning_rate = 0.001
training_iterations = 1000
display_iterations = 200

In [10]:
#Input
X = tf.placeholder(tf.float32,shape=(None,Xfeatures))
#Output
Y = tf.placeholder(tf.float32)

In [12]:
#Neurons
L1 = 3
L2 = 3
L3 = 3

#Layer1 weights
W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1]))
b_fc1 = tf.Variable(tf.constant(0.1,shape=[L1]))

#Layer2 weights
W_fc2 = tf.Variable(tf.random_uniform([L1,L2]))
b_fc2 = tf.Variable(tf.constant(0.1,shape=[L2]))

#Layer3 weights
W_fc3 = tf.Variable(tf.random_uniform([L2,L3]))
b_fc3 = tf.Variable(tf.constant(0.1,shape=[L3]))

#Output layer weights
W_fO= tf.Variable(tf.random_uniform([L3,Yfeatures]))
b_fO = tf.Variable(tf.constant(0.1,shape=[Yfeatures]))

In [13]:
#Layer 1
matmul_fc1=tf.matmul(X, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation
#Layer 2
matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
h_fc2 = tf.nn.relu(matmul_fc2)   #ReLU activation
#Layer 3
matmul_fc3=tf.matmul(h_fc2, W_fc3) + b_fc3
h_fc3 = tf.nn.relu(matmul_fc3)   #ReLU activation
#Output layer
matmul_fc4=tf.matmul(h_fc3, W_fO) + b_fO
output_layer = matmul_fc4  #linear activation

In [14]:
#Loss function
mean_square =  tf.reduce_mean(tf.square(Y-output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square)

#Operation to save variables
saver = tf.train.Saver()

In [17]:
#Initialization and session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={X:InputX1train,Y:InputY1train})
        if i%display_iterations ==0:
            print("Training loss is:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}),"at itertion:",i)
            print("Validation loss is:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}),"at itertion:",i)
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("Final training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))

Training loss: [17.433884]
Training loss is: [17.270817] at itertion: 0
Validation loss is: [15.606103] at itertion: 0
Training loss is: [2.4819336] at itertion: 200
Validation loss is: [2.1471286] at itertion: 200
Training loss is: [1.8077571] at itertion: 400
Validation loss is: [1.6209496] at itertion: 400
Training loss is: [1.3185776] at itertion: 600
Validation loss is: [1.2264187] at itertion: 600
Training loss is: [0.97344327] at itertion: 800
Validation loss is: [0.93670452] at itertion: 800
Model saved in file: /tmp/earthquake_model.ckpt
Final training loss: [0.72876406]
Final validation loss: [0.72199589]


In [18]:
def get_mag(lat, long, depth, date):
    InputX2 = np.asarray([[lat,long,depth,mapdateTotime(date)]],dtype=np.float32)
    InputX2_norm = (InputX2-X1_min)/(X1_max-X1_min)
    InputX1test = np.resize(InputX2_norm,(1,Xfeatures))
    with tf.Session() as sess:
        # Restore variables from disk for validation.
        saver.restore(sess, "/tmp/earthquake_model.ckpt")
        #print("Model restored.")
        #print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))
        #print("output:",sess.run([output_layer],feed_dict={X:InputX1test}))
        return float(sess.run([output_layer],feed_dict={X:InputX1test})[0])

get_mag(50, 50, 0, '10/22/2020')

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt


6.12993860244751

In [29]:
lat = 35
long = 139
df2=pd.read_csv("D:/datafest/tokyo_ep.csv")
#or i in range(5):
   #df2['mag_'+str(i)] = df2.apply(lambda x: get_mag(lat, long, i, x['time']),axis=1)
df2['mag'] = df2['depth'].apply(lambda x: get_mag(lat, long, x, '12/12/2017'))
df2

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

,time,depth,mag
0,11/6/2017,0,6.486719
1,11/7/2017,1,6.490214
2,11/8/2017,2,6.493708
3,11/9/2017,3,6.497203
4,11/10/2017,4,6.500698
5,11/11/2017,5,6.504192
6,11/12/2017,6,6.507687
7,11/13/2017,7,6.511182
8,11/14/2017,8,6.514676
9,11/15/2017,9,6.518171


In [32]:
df2.to_csv('D:/datafest/tokyo_mag.csv')

In [34]:
df3=pd.read_csv("D:/datafest/scale.csv")
#or i in range(5):
   #df2['mag_'+str(i)] = df2.apply(lambda x: get_mag(lat, long, i, x['time']),axis=1)
df3['mag'] = df3.apply(lambda x: get_mag(x['lat'], x['long'], 50, '12/12/2017'),axis=1)
df3

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

,lat,long,mag
0,26,130,6.626652
1,26,131,6.630519
2,26,132,6.634386
3,26,133,6.638252
4,26,134,6.642118
5,26,135,6.645985
6,26,136,6.649852
7,26,137,6.653718
8,26,138,6.657585
9,26,139,6.661452


In [35]:
df3.to_csv('D:/datafest/scale_mag.csv')

In [36]:
df4=pd.read_csv("D:/datafest/time_tokyo.csv")
#or i in range(5):
   #df2['mag_'+str(i)] = df2.apply(lambda x: get_mag(lat, long, i, x['time']),axis=1)
df4['mag'] = df4.apply(lambda x: get_mag(35, 139, 50, x['time']),axis=1)
df4

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

...

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
INFO:tensorflow:Restoring paramete

,time,mag
0,11/6/2017,6.661452
1,11/7/2017,6.661452
2,11/8/2017,6.661452
3,11/9/2017,6.661452
4,11/10/2017,6.661452
5,11/11/2017,6.661452
6,11/12/2017,6.661452
7,11/13/2017,6.661452
8,11/14/2017,6.661452
9,11/15/2017,6.661452


In [19]:
#Testing
lat = input("Enter Latitude between -77 to 86:")
long = input("Enter Longitude between -180 to 180:")
depth = input("Enter Depth between 0 to 700:")
date = input("Enter the date (Month/Day/Year format):")


InputX2 = np.asarray([[lat,long,depth,mapdateTotime(date)]],dtype=np.float32)
InputX2_norm = (InputX2-X1_min)/(X1_max-X1_min)
InputX1test = np.resize(InputX2_norm,(1,Xfeatures))
with tf.Session() as sess:
    # Restore variables from disk for validation.
    saver.restore(sess, "/tmp/earthquake_model.ckpt")
    print("Model restored.")
    #print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))
    print("output:",sess.run([output_layer],feed_dict={X:InputX1test}))

Enter Latitude between -77 to 86:50
Enter Longitude between -180 to 180:50
Enter Depth between 0 to 700:0
Enter the date (Month/Day/Year format):10/22/2020
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
Model restored.
output: [array([[ 6.14258337]], dtype=float32)]


In [20]:
'dwq'+str(9)

'dwq9'